In [ ]:
import requests
from bs4 import BeautifulSoup
import json
import time
from urllib.parse import urljoin

# Define the base URL and starting URL
BASE_URL = "https://mifosforge.jira.com"
START_URL = "https://mifosforge.jira.com/wiki/spaces/projects/overview"

def get_page_content(url):
    headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) "
               "AppleWebKit/537.36 (KHTML, like Gecko) "
               "Chrome/120.0.0.0 Safari/537.36"}
    response = requests.get(url, headers=headers)
    if response.status_code == 200:
        return response.text
    else:
        print(f"Failed to fetch {url}: {response.status_code}")
    return None

def extract_links(html, base_url):
    soup = BeautifulSoup(html, 'html.parser')
    links = set()
    for a in soup.find_all('a', href=True):
        href = a['href']
        full_url = urljoin(base_url, href)
        # Filter to ensure we only follow links within our base domain
        if full_url.startswith(base_url):
            links.add(full_url)
    return list(links)

def extract_data(html):
    soup = BeautifulSoup(html, 'html.parser')
    # Extract visible text
    text = soup.get_text(separator='\n', strip=True)
    
    # Extract hidden content (e.g., elements with display:none)
    hidden_content = []
    for element in soup.find_all(style=True):
        if "display:none" in element.get("style"):
            hidden_content.append(element.get_text(strip=True))
    
    return {"text": text, "hidden": hidden_content}

def scrape_site(start_url, max_urls=5):
    visited = set()
    data = {}
    to_visit = [start_url]
    
    while to_visit and len(visited) < max_urls:
        current_url = to_visit.pop(0)
        if current_url in visited:
            continue
        
        print(f"Scraping: {current_url}")
        html = get_page_content(current_url)
        if html:
            # Extract data from the page and store it with the URL as key
            data[current_url] = extract_data(html)
            # Extract and add new links to the queue
            new_links = extract_links(html, BASE_URL)
            for link in new_links:
                if link not in visited and link not in to_visit:
                    to_visit.append(link)
        visited.add(current_url)
        # Respectful delay between requests
        time.sleep(1)
    
    return data

if __name__ == "__main__":
    # Limit scraping to only 3 URLs (or input number)
    max_urls = 3
    scraped_data = scrape_site(START_URL, max_urls)
    # Save the scraped data in JSON format for later use
    with open("scraped_data.json", "w", encoding="utf-8") as f:
        json.dump(scraped_data, f, ensure_ascii=False, indent=4)


Scraping: https://mifosforge.jira.com/wiki/spaces/projects/overview
Scraping: https://mifosforge.jira.com/wiki/spaces/projects/pages/6816200/Archived+Volunteer+Projects
Scraping: https://mifosforge.jira.com/wiki/spaces/projects/pages/213286919/yemen+info


In [ ]:
!pip install firecrawl-py

In [ ]:
# Install with pip install firecrawl-py
from firecrawl import FirecrawlApp

app = FirecrawlApp(api_key='your api key')

crawl_result = app.crawl_url('https://mifosforge.jira.com/wiki/spaces/projects/overview', params={
'limit': 2, #add limit accordingly to crawl for sublinks
'scrapeOptions': {
	'formats': [ 'markdown' ],
  }
})

print(crawl_result)

{'success': True, 'status': 'completed', 'completed': 2, 'total': 2, 'creditsUsed': 2, 'expiresAt': '2025-03-18T18:17:29.000Z', 'data': [{'markdown': 'Jump to\n\n1. [Confluence navigation](https://mifosforge.jira.com/wiki/spaces/projects/overview#AkTopNav)\n2. [Side navigation](https://mifosforge.jira.com/wiki/spaces/projects/overview#AkSideNavigation)\n3. [Page](https://mifosforge.jira.com/wiki/spaces/projects/overview#AkMainContent)\n\nAtlassian uses cookies to improve your browsing experience, perform analytics and research, and conduct advertising. Accept all cookies to indicate that you agree to our use of cookies on your device. [Atlassian cookies and tracking notice, (opens new window)](https://www.atlassian.com/legal/cookies)\n\nPreferencesOnly necessaryAccept all\n\nProjects\n\n# ![](https://mifosforge.jira.com/wiki/download/attachments/1736705/global.logo?version=2&modificationDate=1427914757501&cacheVersion=1&api=v2)Home\n\nThe Projects space is where we track and share with